# Code first generation of PostgreSQL database

Also need to add a uuid generator function

In [ ]:
# pip install psycopg2

## Create database
To do this first connect to the postgres database

In [ ]:
import psycopg2

params = dict(host="localhost",
    database="postgres",
    user="postgres",
    password="admin",
    port=5432)

params = dict(host="127.0.0.1",
    database="postgres",
    user="postgres",
    password="admin",
    port=5432)


with psycopg2.connect(**params) as con:
    with con.cursor() as cur:
        
        con.autocommit = True
        
        sql = '''DROP DATABASE IF EXISTS pymos_uwa;''';  
        cur.execute(sql)
        
        sql = '''CREATE DATABASE pymos_uwa;''';        
        cur.execute(sql)
            

## Create the fielddata schema
Pretty unnecessary as authentication will be very basic

In [59]:

params = dict(host="127.0.0.1",
    database="pymos_uwa",
    user="postgres",
    password="admin",
    port=5432)

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
        
        cur.execute('CREATE SCHEMA IF NOT EXISTS fielddata')
        

def print_tables():

    """
    Prints the list created by get_tables
    """
        
    with psycopg2.connect(**params) as con:
        with con.cursor() as cur:

            cur.execute("""SELECT table_schema, table_name
                              FROM information_schema.tables
                              WHERE table_schema != 'pg_catalog'
                              AND table_schema != 'information_schema'
                              AND table_type='BASE TABLE'
                              ORDER BY table_schema, table_name""")

            tables = cur.fetchall()
            
            cur.execute("""SELECT table_schema, table_name
                              FROM information_schema.views
                              WHERE table_schema != 'pg_catalog'
                              AND table_schema != 'information_schema'
                              ORDER BY table_schema, table_name""")

            views = cur.fetchall()
            
            print('Tables within of {}/{}'.format(params['host'], params['database']))

            for row in tables:

                print("     {}.{}".format(row[0], row[1]))
                cur.execute("""select column_name from information_schema.columns 
                            where table_schema = '{}' 
                            and table_name='{}'""".format(row[0], row[1]))

                columns = cur.fetchall()
                columns = [column[0] for column in columns]
                print("          Column names: {}\n".format(columns))

            print('Views within of {}/{}'.format(params['host'], params['database']))

            for row in views:

                print("     {}.{}".format(row[0], row[1]))
                cur.execute("""select column_name from information_schema.columns 
                            where table_schema = '{}' 
                            and table_name='{}'""".format(row[0], row[1]))

                columns = cur.fetchall()
                columns = [column[0] for column in columns]
                print("          Column names: {}\n".format(columns))

print_tables()
        

('PostgreSQL 13.3, compiled by Visual C++ build 1914, 64-bit',)
Tables within of 127.0.0.1/pymos_uwa
     fielddata.experiments
          Column names: ['experiment_id', 'experiment_name']

     fielddata.sites
          Column names: ['site_id', 'experiment_id', 'site_name']

Views within of 127.0.0.1/pymos_uwa
     fielddata.sites_view
          Column names: ['experiment_name', 'site_name']



### Add the auto uuid generator extension

In [60]:
sql='CREATE EXTENSION IF NOT EXISTS "uuid-ossp";'

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:

        cur.execute(sql)

        con.commit()
        
sql='SELECT uuid_generate_v1();'

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:

        cur.execute(sql)

        con.commit()

## Create the experiments table
Going to have an autogenerated UUID as a primary key and a unique experiment_name

In [61]:
sql = ['DROP TABLE IF EXISTS fielddata.experiments CASCADE;',]

sql += ['''CREATE TABLE IF NOT EXISTS fielddata.experiments 
(
  experiment_id UUID NOT NULL DEFAULT uuid_generate_v1() , 
  experiment_name CHARACTER (10) NOT NULL , 
  CONSTRAINT pkey_experiments PRIMARY KEY ( experiment_id ),
  CONSTRAINT ukey_experiments UNIQUE ( experiment_name )
)''',
     ]

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:
        
        for s in sql:
            cur.execute(s)

            con.commit()

## Create the sites and deployment data tables
### Sites table
Going to have an autogenerated UUID as a primary key and a unique constraint on the combination of site_name and experiment_name.

For now we can make the foreign key to the fielddata.experiments(experiment_name) column, although it's not a primary key.

### Deployment table

This is going to be very basic. It will contain only the data that was in mrayson's excel sheet. This should be updated to include:
- deployment field trip
- collection field trip 
- deployment type [e.g. mooring, profile]
- process level [e.g. raw, converted_in_vector]
- actual foreign key to the instruments table, not just live fields

Currently as there is no field trip, trip can't be in the unique constraint. The uinique constraint would be overly tough for repeat deployments in a single experiment.  

but at the moment it does not!  

In [135]:
sql = ['DROP TABLE IF EXISTS fielddata.sites CASCADE;',]

sql += ['''CREATE TABLE IF NOT EXISTS fielddata.sites 
(
  site_id UUID NOT NULL DEFAULT uuid_generate_v1() , 
  experiment_ID UUID references fielddata.experiments(experiment_id), 
  site_name CHARACTER (10) NOT NULL , 
  latitude NUMERIC NOT NULL , 
  longitude NUMERIC NOT NULL , 
  depth NUMERIC NULL , 
  CONSTRAINT pkey_sites PRIMARY KEY ( site_id ),
  CONSTRAINT ukey_sites UNIQUE ( experiment_ID, site_name )
)''',
     ]

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:
        
        for s in sql:
            cur.execute(s)

            con.commit()
            
sql = ['DROP TABLE IF EXISTS fielddata.deployments CASCADE;',]

sql += ['''CREATE TABLE IF NOT EXISTS fielddata.deployments 
(
  deployments_id UUID NOT NULL DEFAULT uuid_generate_v1() , 
  site_id UUID references fielddata.sites(site_id), 
  instrument CHARACTER (20) NOT NULL , 
  serial_number CHARACTER (20) NOT NULL , 
  deployment_type CHARACTER (20) NOT NULL , 
  height_nominal NUMERIC NULL , 
  height_actual NUMERIC NULL , 
  filename CHARACTER (50) NOT NULL , 
  folder CHARACTER (250) NOT NULL ,
  CONSTRAINT pkey_deployments PRIMARY KEY ( deployments_id ),
  CONSTRAINT ukey_deployments UNIQUE ( site_id, instrument, serial_number ),
  CONSTRAINT allowed_deployment_type
  CHECK (deployment_type IN (
    'mooring', -- moored deployment
    'profile', -- profiling deployment
    'drift' -- drifting or drogue deployment
  ))
)''',
     ]

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:
        
        for s in sql:
            cur.execute(s)

            con.commit()


print_tables()

Tables within of 127.0.0.1/pymos_uwa
     fielddata.deployments
          Column names: ['deployments_id', 'site_id', 'instrument', 'serial_number', 'deployment_type', 'height_nominal', 'height_actual', 'filename', 'folder']

     fielddata.experiments
          Column names: ['experiment_id', 'experiment_name']

     fielddata.sites
          Column names: ['site_id', 'experiment_id', 'site_name', 'latitude', 'longitude', 'depth']

Views within of 127.0.0.1/pymos_uwa


## Insert some data
Insert two expriments experiments table.

Can't insert sites yet because I don't know the ID that the DB generated for the trips.

In [136]:

sql = """INSERT INTO fielddata.experiments(experiment_name) VALUES('{}')
                        ON CONFLICT ON CONSTRAINT ukey_experiments 
                        DO NOTHING;"""

sites = ['KISSME', 'RS19']
with psycopg2.connect(**params) as con:
    with con.cursor() as cur:

        for site in sites:
            cur.execute(sql.format(site))

            con.commit()
            
# sql = "INSERT INTO fielddata.sites(experiment_name, site_name) VALUES('KISSME', '{}')"

# sites = ['NP250', 'SP250', 'SP250L', 'WP250']
# with psycopg2.connect(**params) as con:
#     with con.cursor() as cur:

#         for site in sites:
#             cur.execute(sql.format(site))

#             con.commit()

# sql = "INSERT INTO fielddata.sites(experiment_name, site_name) VALUES('RS19', '{}')"

# sites = ['SP250', 'T300', 'T220', 'T150', 'W150', 'L150']
# with psycopg2.connect(**params) as con:
#     with con.cursor() as cur:

#         for site in sites:
#             cur.execute(sql.format(site))

#             con.commit()


## Create a view


In [137]:
sql = ['''DROP VIEW IF EXISTS fielddata.sites_view''']

sql += ['''CREATE OR REPLACE VIEW fielddata.sites_view AS
          SELECT
            fielddata.experiments.experiment_name,
            fielddata.sites.site_name,
            fielddata.sites.longitude,
            fielddata.sites.latitude,
            fielddata.sites.depth
            FROM fielddata.sites
            INNER JOIN fielddata.experiments ON fielddata.experiments.experiment_id = fielddata.sites.experiment_id''']

with psycopg2.connect(**params) as con:
    with con.cursor() as cur:
        
        for s in sql:
            cur.execute(s)

            con.commit()

def print_sites():
    print('\nPrinting sites table...')
    with psycopg2.connect(**params) as con:
        with con.cursor() as cur:

            cur.execute('SELECT * FROM fielddata.sites')

            # display the PostgreSQL database server version
            outputs = cur.fetchall()
            for output in outputs:
                print(output)
                
def print_deployments():
    print('\nPrinting deployments table...')
    with psycopg2.connect(**params) as con:
        with con.cursor() as cur:

            cur.execute('SELECT * FROM fielddata.deployments')

            # display the PostgreSQL database server version
            outputs = cur.fetchall()
            for output in outputs:
                print(output)
                
def print_sites_view():
    print('\nPrinting sites view...')
    with psycopg2.connect(**params) as con:
        with con.cursor() as cur:

            cur.execute('SELECT * FROM fielddata.sites_view')

            # display the PostgreSQL database server version
            outputs = cur.fetchall()
            for output in outputs:
                print(output)

def print_db():
    
    print_tables()
    print_sites()
    print_sites_view()
    print_deployments()
    
    
print_db()

Tables within of 127.0.0.1/pymos_uwa
     fielddata.deployments
          Column names: ['deployments_id', 'site_id', 'instrument', 'serial_number', 'deployment_type', 'height_nominal', 'height_actual', 'filename', 'folder']

     fielddata.experiments
          Column names: ['experiment_id', 'experiment_name']

     fielddata.sites
          Column names: ['site_id', 'experiment_id', 'site_name', 'latitude', 'longitude', 'depth']

Views within of 127.0.0.1/pymos_uwa
     fielddata.sites_view
          Column names: ['experiment_name', 'site_name', 'longitude', 'latitude', 'depth']


Printing sites table...

Printing sites view...

Printing deployments table...


In [138]:

def add_site(experiment_name, site_data, verbose=False):

    with psycopg2.connect(**params) as con:
        with con.cursor() as cur:

            sql = "SELECT experiment_id FROM fielddata.experiments where experiment_name = '{}'"
            cur.execute(sql.format(experiment_name))
            
            experiment_id = cur.fetchall()[0][0]
#             for output in outputs:
            if verbose:
                print(experiment_id)
                
            for row in site_data:

                sql = """INSERT INTO fielddata.sites (experiment_id, site_name, latitude, longitude, depth) 
                        VALUES ('{}', '{}', '{}', '{}', '{}')
                        ON CONFLICT ON CONSTRAINT ukey_sites 
                        DO NOTHING;"""
                cur.execute(sql.format(experiment_id, *row))

            con.commit()
            
RS19 = [['NP250', 123.3535, -13.7621, -252],
        ['T300', -1, -1, -300],
        ['T220', -1, -1 , -220],
        [ 'T150', -1, -1, -150],
        [ 'W150', -1, -1, -150],
        [ 'L150', -1, -1, -150]]
KISSME = [['NP250', 123.3535, -13.7621, -252], 
          ['SP250', 123.3542, -13.7669, -252],
          ['WP250', 123.3539, -13.7681, -252],
          ['SP250L', 123.349, -13.7659, -252]]

add_site('RS19', RS19)
add_site('KISSME', KISSME)

print_db()


Tables within of 127.0.0.1/pymos_uwa
     fielddata.deployments
          Column names: ['deployments_id', 'site_id', 'instrument', 'serial_number', 'deployment_type', 'height_nominal', 'height_actual', 'filename', 'folder']

     fielddata.experiments
          Column names: ['experiment_id', 'experiment_name']

     fielddata.sites
          Column names: ['site_id', 'experiment_id', 'site_name', 'latitude', 'longitude', 'depth']

Views within of 127.0.0.1/pymos_uwa
     fielddata.sites_view
          Column names: ['experiment_name', 'site_name', 'longitude', 'latitude', 'depth']


Printing sites table...
('a2824246-c365-11eb-9ab9-34cff6b1008e', '08e87b06-c355-11eb-946c-34cff6b1008e', 'NP250     ', Decimal('123.3535'), Decimal('-13.7621'), Decimal('-252'))
('a283f26c-c365-11eb-9aba-34cff6b1008e', '08e87b06-c355-11eb-946c-34cff6b1008e', 'T300      ', Decimal('-1'), Decimal('-1'), Decimal('-300'))
('a283fe56-c365-11eb-9abb-34cff6b1008e', '08e87b06-c355-11eb-946c-34cff6b1008e', 'T220  

In [141]:
def add_deployment(experiment_name, site_name, deployment_data, verbose=False):

    with psycopg2.connect(**params) as con:
        with con.cursor() as cur:

            sql = "SELECT experiment_id FROM fielddata.experiments where experiment_name = '{}'"
            cur.execute(sql.format(experiment_name))
            experiment_id = cur.fetchall()[0][0]
            
            sql = """SELECT site_id FROM fielddata.sites where 
                    experiment_id = '{}'
                    and site_name = '{}'"""
            
            cur.execute(sql.format(experiment_id, site_name))
            site_id = cur.fetchall()[0][0]
            
            if verbose:
                print(experiment_id)
                print(site_id)
                
            for row in deployment_data:

                sql = """INSERT INTO fielddata.deployments (
                                site_id,
                                instrument,
                                serial_number,
                                deployment_type,
                                height_nominal,
                                height_actual,
                                filename,
                                folder) 
                        VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')
                        ON CONFLICT ON CONSTRAINT ukey_deployments 
                        DO NOTHING;"""
                cur.execute(sql.format(site_id, *row))
    
            con.commit()

# 'instrument', 'serial_number', 'deployment_type', 'height_nominal', 'height_actual', 'filename', 'folder'
deployment_data = [['SBE39 TP', '6527', 'mooring', 15.4, 15.4, '6527.asc', 'Data\SBE39']]
add_deployment('RS19', 'NP250', deployment_data)

print_db()


Tables within of 127.0.0.1/pymos_uwa
     fielddata.deployments
          Column names: ['deployments_id', 'site_id', 'instrument', 'serial_number', 'deployment_type', 'height_nominal', 'height_actual', 'filename', 'folder']

     fielddata.experiments
          Column names: ['experiment_id', 'experiment_name']

     fielddata.sites
          Column names: ['site_id', 'experiment_id', 'site_name', 'latitude', 'longitude', 'depth']

Views within of 127.0.0.1/pymos_uwa
     fielddata.sites_view
          Column names: ['experiment_name', 'site_name', 'longitude', 'latitude', 'depth']


Printing sites table...
('a2824246-c365-11eb-9ab9-34cff6b1008e', '08e87b06-c355-11eb-946c-34cff6b1008e', 'NP250     ', Decimal('123.3535'), Decimal('-13.7621'), Decimal('-252'))
('a283f26c-c365-11eb-9aba-34cff6b1008e', '08e87b06-c355-11eb-946c-34cff6b1008e', 'T300      ', Decimal('-1'), Decimal('-1'), Decimal('-300'))
('a283fe56-c365-11eb-9abb-34cff6b1008e', '08e87b06-c355-11eb-946c-34cff6b1008e', 'T220  

In [143]:
pip install django

In [145]:
import django
from django import forms

from django import forms
from django.core.exceptions import ValidationError
from django.utils.translation import ugettext_lazy as _

class RenewBookForm(forms.Form):
    renewal_date = forms.DateField(help_text="Enter a date between now and 4 weeks (default 3).")

    def clean_renewal_date(self):
        data = self.cleaned_data['renewal_date']

        # Check if a date is not in the past.
        if data < datetime.date.today():
            raise ValidationError(_('Invalid date - renewal in past'))

        # Check if a date is in the allowed range (+4 weeks from today).
        if data > datetime.date.today() + datetime.timedelta(weeks=4):
            raise ValidationError(_('Invalid date - renewal more than 4 weeks ahead'))

        # Remember to always return the cleaned data.
        return data